# Cargamos las ejecuciones de la tarea anterior

In [1]:
import json
import os
import pandas as pd
import numpy as np

In [2]:
os.chdir("../PEC3/representaciones")
path = os.getcwd()
representations_files = []
for file in os.listdir(path):
    representations_files.append(path + "/"+file)
representations_files

['/Users/maciasalvasalva/Desktop/UNED/Descubrimiento de informacion en textos/Entregas/PEC3/representaciones/word2vec_representation.json',
 '/Users/maciasalvasalva/Desktop/UNED/Descubrimiento de informacion en textos/Entregas/PEC3/representaciones/additive_representation.json',
 '/Users/maciasalvasalva/Desktop/UNED/Descubrimiento de informacion en textos/Entregas/PEC3/representaciones/mean_representation.json',
 '/Users/maciasalvasalva/Desktop/UNED/Descubrimiento de informacion en textos/Entregas/PEC3/representaciones/tfidf_representation.json']

In [3]:
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# Carga los datos
tfidf_data = load_json(representations_files[3])
word2vec_data = load_json(representations_files[0])
additive_data = load_json(representations_files[1])
mean_data = load_json(representations_files[2])

In [8]:
# word2vec_data